## Mini-batch Gradient Descent

- Big data helped DL thrive.
- Minibatching is an efficient way to train DL models on big data.

What if you have 5 million samples in your data? It will take sooo long to take one single step in gradient descent

Instead use batches of size 1k. thats all

Batch gradient descent means on all 5 million examples at once.

Mini-batch means on 1k examples at a time.


1 epoch means a single pass over the whole training set. So in the above example, one epoch consists of 5k iterations.

## Understanding Mini-batch Gradient Descent

Learn more details why it works.

if m=1 then it is named "Stochastic Gradient Descent". So we take gradient descent with one sample at a time.


In practice, we use something in between 1 and m (two extremes).

If we use size=m: Too long per iteration
If we use size=1 (SGD): Huge disadvantage because lose speed up that would come from vectorization.

So we use in-between, i.e., mini-batch:
- Get benefits of vectorization
- Make progress without needing to wait to process the entire training set. 

Mini-batch gradient descent also will never fully converge but always oscillate around the minimum region (that is because mini-batch never identical, 100% representative of the full training set). You can mitigate this by decaying the learning rate.


### So how to choose what size is optimal for the mini-batch? 

Some guidelines:

- If small training set: Just use batch gradient descent
    - Around lets say 2k samples
- Typical mini-batch sizes: 64,128,256,512 (power of 2 is good rule of thumb)
- Make sure mini-batch fits in the CPU/GPU memory. Adjust the size appropriately.
- This itself is a hyperparam, which is something we can tune

## Exponentially Weighted Moving Averages
(More efficient ways of optimization then batch or mini-batch GD).

Temperature in London example: moving average idea to remove the noise from each sample.
for $\beta=0.9$
$$
V(t) = 0.9 V(t-1) + 0.1 * \Theta_t 
$$

where $\Theta_t$ is the measure temperature of that day.

Doing so is equivalent to taking average over $\dfrac{1}{1-\beta}$ days. So for the example above, it is taking average over the last ten days (not exactly average but it is weighted average of course).


As $\beta$ approaches to 1 it will get smoother and smoother.

## Understanding Exponentially Weighted Averages

So using the basic formula above we can try to expand the equation several steps to see the recursive pattern:

$$
V_{100} = 0.1 \Theta_{100} + 0.9 * 0.1 * \Theta_{99} + 0.9^2 * 0.1 * \Theta_{98} + 0.9^3 * 0.1 * \Theta_{97} + ....
$$

Which when you think about it is an elementise product of two vectors: 

$$
vector_1 = [\Theta_{100}, \Theta_{99},\Theta_{98},... , \Theta_{1}] \\
vector_2 = 0.1 * [0.9^0 , 0.9^1 , 0.9^2,..., 0.9^{99} ]
$$


Next question: So in practice how many days does this average over?

(Rough) Answer, we can use one important mathematical equation to help us have approximate answer:

$$
(1-\epsilon)^{1/\epsilon} = \dfrac{1}{e}
$$

Using this equation we can say that if $\beta=0.9$ then $(1-0.1)^{1/0.1}=\dfrac{1}{e}$

So the effect of ten days prior gets to ~ one third compared to current day. So we can say around $\dfrac{1}{1-\beta}$ days is averaged in the above exponentially weighted averaging.

For example if $\beta=0.98$ then we need $50$ days for the effect to get to one third.

IMPORTANT: Above is just an approximation not a mathematically correct statement. 

**How to implement it actually?**


In [29]:
# Memory efficient 
beta = 0.5
thetas = [1,2,3,4,5,6,7,8,9,10] #imagine streaming
V_t = 0
# Repeat
for theta in thetas:
    V_t = beta * V_t + (1-beta)* theta


In [30]:
V_t

9.0009765625

In [14]:
import numpy as np
np.mean(thetas)

5.5

## Bias Correction
Last part of the puzzle we need to build a more efficient gradient descent optimization.


If we use the above implementation, we start with very low values since for example for the first day (since we don't have samples before it) we get 0.1 times the value of the first sample.

Something like a cold start problem.


Solution: we divide by $(1-\beta^{t})$ to correct for small values of $t$ (beginning of the samples):

So $V_t$ becomes:

$$
V_t = \dfrac{V_t}{1-\beta^{t}}
$$

notice that the denominator will converge to 1 since $\beta^t$ will converge to 0 for large values of t.

Some people do not bother implementing this bias correction and just wait for the gradient descent to warmup but if you want to be very accurate bias correction is calculated as above!!

 

## Gradient descent with momentum

One sentence description: Compute an exponentially weighted average of your gradients.


Momentum algorithm.

On iteration t: 
- Compute dW dB on current minibatch
- $V_{dw} = \beta V_{dw} + (1-\beta) dW$
- $V_{db} = \beta V_{db} + (1-\beta) db$
- Finally use $V_{dw}$ and $V_{db}$ as your gradients for update:

$$
w = w - \alpha V_{dw}\\
b = b - \alpha V_{db}\\
$$

where $\alpha$ is the learning rate.


$\beta=0.9$ is commonly used. And people don't even bother with the bias correction since it will correct itself in ten iterations


**Intuition1.** Imagine an elliptic loss contour (vertically shrinked and long horizontally). At every iteration we want to take small steps along vertical axis and large steps along the horizontal axis.


If gradient descent is oscillating up and down, using the above momentum formula will help us average out the up and down movements and at each iteration we will make a large horizontal step and a tiny vertical step. Something like course correction.


**Intuition2.** Physics way: It is like $dW$ and $dB$ are acceleration and $V_{dW}$ and $V_{db}$ are velocity. So the gradients have influence on the final speed but can not immediately change it. If velocity was low along one direction, adding acceleration (with multiplied by $1-\beta$) will not speed up the ball in that direction right away.

## RMSprop

Another way to speed up gradient descent is to use RMSprop.

The intuition behind it is that, if we have (in the hyperdimensional space) a large oscillation at each iteration in a direction, we want to slow it down. So we divide the gradients by the rolling average vector that we calculate (similar to momentum). This helps smooth our gradient vectors and converge faster.

**How to implement:**
For sake of simplicity lets assume we have two weights $w$ and $b$.


- Calculate the weighted averages:
 - $S_{dw}=\beta S_{dw} + (1-\beta) dw^{2}$
 - $S_{db}=\beta S_{db} + (1-\beta) db^{2}$
- Apply gradient descent in rms prop style:
 - $w = w - \alpha \dfrac{dw}{\sqrt{S_{dw}}}$
 - $b = b - \alpha \dfrac{db}{\sqrt{S_{db}}}$
 
This means that if the oscillation along $b$ was large, we diminished this effect by dividing it with something large. This way we can use a relatively larger learning rate $\alpha$ to speed up convergence without overshooting across some dimension.

In the next step, we will actually combine RMSProp and momentum when updating the gradients. 

Finally, for numerical stability we actually add a small number $\epsilon$ to the denominators so that they don't blow up if the $\sqrt{S_{x}}$ are almost 0:

 - $w = w - \alpha \dfrac{dw}{\sqrt{S_{dw}}+\epsilon}$
 - $b = b - \alpha \dfrac{db}{\sqrt{S_{db}}+\epsilon}$


Fun fact: Geoffrey Hinton introduced RMSProp in a coursera course for the first time and made it popular actually! hehe so funny..

## Adam Optimization Algorithm

There have been a plethora of optimization algorithms proposed by machine learning scientists but most of them failed to generalize to other network architectures... 

So, the community is usually skeptic to new algorithms but RMSProp and Adam are one of those that stood the test of time!!!


**Adam Optimization Algorithm**
- Initialize to zero: $V_{dw}, V_{db}, S_{dw}, S_{db}$


- Calculate momentum values $V_{dw}$ and $V_{db}$:

$$
V_{dw} = \beta_1 V_dw+ (1-\beta_1) dw\\
V_{db} = \beta_1 V_db+ (1-\beta_1) db
$$


- Calculate RMSProp values $S_{dw}$ and $S_{db}$:
$$
S_{dw} = \beta_2 S_dw+ (1-\beta_2) dw^2\\
S_{db} = \beta_2 S_db+ (1-\beta_2) db^2
$$

- Apply bias correction (for both Vs and Ss): 

$$
V_{dw}= \dfrac{V_{dw}}{1-\beta_1^t}\\
...
$$


- Then update weights by combining the two as below:

$$
w = w - \alpha \dfrac{V_{dw}}{\sqrt{S_dw}+\epsilon} \\ 
b = b - \alpha \dfrac{V_{db}}{\sqrt{S_db}+\epsilon} \\ 
$$


where the numerator is coming from the momentum formula and denominator is coming from the RMSProp formula.
Note $\epsilon$ in the above equations is outside of square root!!


Total hyperparameters: 
- $\alpha$ needs to be tuned 
- $\beta_1$: 0.9 ($dw$)
- $\beta_2$: 0.999 ($dw^2$)
- $epsilon$: $10^{-8}$

So where does the name "ADAM" come from?

It means "adaptive moment estimation" since $dw$ is the first moment and $dw^2$ is the second moment of the gradients.

<a name='5'></a>   
## (From Assignment) 5 - Adam

Adam is one of the most effective optimization algorithms for training neural networks. It combines ideas from RMSProp (described in lecture) and Momentum. 

**How does Adam work?**
1. It calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v^{corrected}$ (with bias correction). 
2. It calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s^{corrected}$ (with bias correction). 
3. It updates parameters in a direction based on combining information from "1" and "2".

The update rule is, for $l = 1, ..., L$: 

$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$
where:
- t counts the number of steps taken of Adam 
- L is the number of layers
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages. 
- $\alpha$ is the learning rate
- $\varepsilon$ is a very small number to avoid dividing by zero

As usual, all parameters are stored in the `parameters` dictionary  

## Learning Rate Decay 

One thing that might speed up the learning algorithm is to use learning decay.

Why? Because when you get closer and closer to the global minimum you should be making more fine-grained, small moves. If you have big steps next to the global minimum, you will keep oscillating.

In [39]:
# One implementatiopn
current_epoch_num = 12
decay_rate = 0.1
initial_learning_rate = 0.1

def decay(decay_rate, epoch_num,initial_learning_rate):
    return 1/ (1+decay_rate * epoch_num) * initial_learning_rate

In [36]:
decay(0.1,100,0.1)

0.009090909090909092

In [37]:
decay(1,100,0.1)

0.0009900990099009901

Another approach is to use **exponential decay**.



In [38]:
def exponential_decay(decay_rate, epoch_num,initial_learning_rate):
    """returns the learning rate"""
    return decay_rate**epoch_num * initial_learning_rate

Another approach if you are training model that takes very long is to _Manual decay_:

- Observe the loss values
- Manually set the new learning rate


According to Andrew, tuning the learning rate is way more higher on the list of things he would tune compared to focusing on the learning rate decay tuning. As long as you have a good learning rate, decay is apparently less important.

## The problem of Local Optima

Are local optima really problem in hyperdimensional spaces with lets say 1k dimensions???

Not really! That is because a conventional local optima assumes that across all dimensions the point is the optimal value. However that is super unlikely to be the case for high dimensional spaces with 1k dimensions.

What is more common is `plateaus` and `saddle points`!!

- Saddle point is when a point is local minimum in terms of one dimension and local maximum in another dimension. Such a point is not an issue for gradient descent because it will get out of the saddle point by following the dimension where we had the local maxima (concave shape)

- Plateaus are regions where the gradient is very low. This is real problem and might take a lot for your algorithm to get out of it.


The fact that local optima is not a serious problem in very high dimensional space is an important lesson in machine learning. Historically we worked on 3-5 dimensions (low-dimensional spaces) and local optima was a serious problem with no clear solution. However such insights do not transfer to very high dimensional spaces.